In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
def mine_level_temporal_aggregation(
    objects_gdf,
    smooth_window=3,
    export_csv_path=None,
    make_plots=True
):
    """
    STEP 6 — Mine-Level Temporal Aggregation

    Parameters
    ----------
    objects_gdf : GeoDataFrame
        Must contain:
        - date (datetime)
        - start_date (datetime)
        - area_ha (float)
    smooth_window : int
        Rolling window for smoothing cumulative area
    export_csv_path : str or None
        If provided, saves CSV
    make_plots : bool
        Whether to generate plots

    Returns
    -------
    mine_timeseries : DataFrame
        Aggregated mine-level temporal metrics
    """

    # -------------------------------
    # GROUP A — Sanity & preparation
    # -------------------------------
    df = objects_gdf.copy()

    df["date"] = pd.to_datetime(df["date"])
    df["start_date"] = pd.to_datetime(df["start_date"])

    dates = sorted(df["date"].unique())

    # -----------------------------------------
    # GROUP B — Total excavated area A_mine(t)
    # -----------------------------------------
    total_excavated_area = []

    for t in dates:
        area_t = df.loc[
            df["start_date"] <= t, "area_ha"
        ].sum()
        total_excavated_area.append(area_t)

    total_excavated_area = pd.Series(
        total_excavated_area,
        index=dates,
        name="total_excavated_area_ha"
    )

    # -----------------------------------------
    # GROUP C — New excavation ΔA_new(t)
    # -----------------------------------------
    new_excavation = (
        df.groupby("start_date")["area_ha"]
        .sum()
        .reindex(dates, fill_value=0.0)
    )
    new_excavation.name = "new_excavation_ha"

    # -----------------------------------------
    # GROUP D — Recovery (explicitly zero)
    # -----------------------------------------
    recovered_area = pd.Series(
        0.0, index=dates, name="recovered_ha"
    )

    # -----------------------------------------
    # GROUP E — Build time series table
    # -----------------------------------------
    mine_timeseries = pd.DataFrame({
        "date": dates,
        "total_excavated_area_ha": total_excavated_area.values,
        "new_excavation_ha": new_excavation.values,
        "recovered_ha": recovered_area.values
    })

    # -----------------------------------------
    # GROUP F — Net change & rate
    # -----------------------------------------
    mine_timeseries["net_change_ha"] = (
        mine_timeseries["total_excavated_area_ha"]
        .diff()
        .fillna(0.0)
    )

    mine_timeseries["excavation_rate_ha_per_step"] = (
        mine_timeseries["new_excavation_ha"]
    )

    # -----------------------------------------
    # GROUP G — Optional smoothing
    # -----------------------------------------
    mine_timeseries["total_excavated_area_smooth"] = (
        mine_timeseries["total_excavated_area_ha"]
        .rolling(
            window=smooth_window,
            center=True,
            min_periods=1
        )
        .median()
    )

    # -----------------------------------------
    # GROUP H — Visualization
    # -----------------------------------------
    if make_plots:
        # Total excavated area
        plt.figure(figsize=(9, 4))
        plt.plot(
            mine_timeseries["date"],
            mine_timeseries["total_excavated_area_ha"],
            marker="o",
            label="Total Excavated Area"
        )
        plt.plot(
            mine_timeseries["date"],
            mine_timeseries["total_excavated_area_smooth"],
            linewidth=2,
            label="Smoothed"
        )
        plt.xlabel("Date")
        plt.ylabel("Area (ha)")
        plt.title("Total Excavated Area Within Legal Mine Boundary")
        plt.legend()
        plt.grid(True)
        plt.show()

        # New excavation
        plt.figure(figsize=(9, 4))
        plt.bar(
            mine_timeseries["date"],
            mine_timeseries["new_excavation_ha"],
            width=10
        )
        plt.xlabel("Date")
        plt.ylabel("New Excavation (ha)")
        plt.title("New Excavation Detected Per Date")
        plt.grid(True)
        plt.show()

    # -----------------------------------------
    # GROUP I — Export
    # -----------------------------------------
    if export_csv_path is not None:
        mine_timeseries.to_csv(export_csv_path, index=False)

    return mine_timeseries


In [ ]:
# Example usage
mine_ts = mine_level_temporal_aggregation(
    objects_gdf,
    smooth_window=3,
    export_csv_path="mine_level_excavation_timeseries.csv",
    make_plots=True
)

mine_ts.head()
